In [56]:
import numpy as np
import pandas as pd
import re

In [57]:
df = pd.read_excel('TRR 3.xlsx')

#The data is organized by 
Lessee: NA
Lessor: well data
There are breaks occasionally in the data with long numbers in them, these might be years
Eg)
70001800100000
77421230000000
1.31992021400208E+27
79513800100000

Pretty sure 0000000000 is a signal for a line break
Eg)
0801Z AMENDED FIELD NAME, TOTAL DEPTH, AND TYPE OF OPERATION.               P0000000000
0802Z PERMIT PERVIOUSLY AMENDED 9-12-77.DIRECTIONAL DRILL LETTER ATTACHED   P0000000000
1200120040803PRIOR TO 9/1/1986 ONE LOCATION DATA ENTRY SCREEN EXISTED, AND FOR     0000000000
1200220040803DIRECTIONALLY DEVIATED WELLS, ONLY THE BOTTOM HOLE LOCATION WAS KEYED.0000000000
1200320040803THEREFORE, WHAT IS DISPLAYED AS A SURFACE LOCATION FOR THIS PERMIT    0000000000
^ They also add comments like this all over the file

In [58]:
## Take the first two columns so it is a dataframe object
def make_df2():
    df2 = df[['1200119870303W-1 FILED AS AN AMENDED NEW STATU','Unnamed: 2']]
    df2 = df2.rename(columns = {'1200119870303W-1 FILED AS AN AMENDED NEW STATU':'Col1', 'Unnamed: 2': 'Col2'})
    return df2
df2 = make_df2()

In [59]:
df2.head()

,Col1,Col2
0,1200219870303PERMIT 324685. (BJM) 03/03/87,NaN
1,"01000000199175O'CONNOR, T. J. (JIM)",00ANNNNNNNNNNN00000011977060600000000N ...
2,"02000000199175O'CONNOR, T. J. (JIM)",0564337702700000013880001977060619770606000000...
3,0319028076GG N00000000197703040000000000000000,NaN
4,01000000299321STATE OF TEXAS TRACT 296,00ANNNNNNNNNNN00000021977060600000000N ...


In [60]:
## Making sure there is a string in there
type(df2['Col1'][1])

str

In [61]:
## Split the first column in two. There are 14 digits followed by a tract. This will return a column, one with everything after the first 14 digits.
df2 = df2.Col1.str.split(pat = '\d{14}', n = 5, expand = True)

In [62]:
#Create a column in the original df with the output from above
df['tract_owner'] = df2[1]

In [63]:
#Now to get the first 14 digits
df2 = make_df2()
df2.head()
df2['Col3'] = df2['Col1'].str[:14]

In [64]:
#Function to apply regex to column
def remove_char(s):
    return re.sub('[^0-9]+', '', s)
def remove_zero(s):
    return re.sub('0','',s)
def remove_N(s):
    return re.sub('N', '', s)

In [65]:
#Change the values of a column to a string, then apply the regex
df2['Col3'] = df2['Col3'].apply(str)
df2['Col3'] = df2['Col3'].apply(remove_char)
df['tract_id'] = df2['Col3']

In [66]:
#Getting the Numbers from Column B, we think its a date
df['S NUMBER ASSIGNED IN ERROR. REFER TO 0000000000'] = df['S NUMBER ASSIGNED IN ERROR. REFER TO 0000000000'].apply(str)
df['ID_Date_B'] = df['S NUMBER ASSIGNED IN ERROR. REFER TO 0000000000'].apply(remove_char)

In [67]:
#Getting the stuff after the numbers in B
df['S NUMBER ASSIGNED IN ERROR. REFER TO 0000000000'] = df['S NUMBER ASSIGNED IN ERROR. REFER TO 0000000000'].apply(str)
df['firm_name'] = df['S NUMBER ASSIGNED IN ERROR. REFER TO 0000000000'].str[20:]

In [68]:
#Getting the Numbers from Column C
df2['ID_Date'] = df2['Col2'].apply(str)
df2['ID_Date'] = df2['ID_Date'].apply(remove_zero)
df2['ID_Date'] = df2['ID_Date'].str[12:]
#Adding the numbers from column C to a new column
#df['ID_Date_ColC'] = df2['ID_Date']

In [69]:
df2

,Col1,Col2,Col3,ID_Date
0,1200219870303PERMIT 324685. (BJM) 03/03/87,NaN,1200219870303,
1,"01000000199175O'CONNOR, T. J. (JIM)",00ANNNNNNNNNNN00000011977060600000000N ...,01000000199175,1197766N 1
2,"02000000199175O'CONNOR, T. J. (JIM)",0564337702700000013880001977060619770606000000...,02000000199175,8197766197766 197866
3,0319028076GG N00000000197703040000000000000000,NaN,0319028076,
4,01000000299321STATE OF TEXAS TRACT 296,00ANNNNNNNNNNN00000021977060600000000N ...,01000000299321,2197766N 1
...,...,...,...,...
1048570,0330655400OO N00000000198307240000000000000000,NaN,0330655400,
1048571,0601D 0933 1200 P0000,NaN,06010933120,
1048572,0602C P0000,NaN,0602,
1048573,0603W 01750 P0000,NaN,060301750,


In [71]:
#Get survey name
df['Unnamed: 3'] = df['Unnamed: 3'].apply(str)
df['Survey'] = df['Unnamed: 3'].str[19:]

In [72]:
#What looks like coordinates
df['Unnamed: 4'] = df['Unnamed: 4'].apply(str)
df['Coordinates'] = df['Unnamed: 4'].str[32:]

In [73]:
#Rename the city variable
df['Cities'] = df['Unnamed: 5']

In [74]:
#Well location?
df['Well_location'] = df['Unnamed: 6']

In [75]:
# Well location but now with zeros
df['Other_well_location'] = df['Unnamed: 7'].str[14:]

In [85]:
#Coordinates, have commas
df['Unnamed: 8'] = df['Unnamed: 8'].apply(str)
df['Coordinates_2'] = df['Unnamed: 8'].str[14:]

In [88]:
#Check to see if unnamed: 9its only O00000000
df['Unnamed: 9'] = df['Unnamed: 9'].apply(str)
df.loc[df['Unnamed: 9'] == 'O00000000', 'nine'] = 0
df['nine'].sum()
#value of 0 means that every value in the column was O00000000, all other values were N/A and not counted

0.0

In [118]:
#More location things, perhaps some ID. There are two strings, and 10 zeros between them. 
df3 = df['Unnamed: 10'].str.split(pat = '0000000000', expand = True)
df3 = df3.rename(columns = {0:'first', 1:'second'})
df3
df['ten_one'] = df3['first']
df['ten_two'] = df3['second']

In [119]:
df.columns

Index(['1200119870303W-1 FILED AS AN AMENDED NEW STATU',
       'S NUMBER ASSIGNED IN ERROR. REFER TO 0000000000', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'tract_owner', 'tract_id',
       'ID_Date_B', 'firm_name', 'Survey', 'Coordinates', 'Cities',
       'Well_location', 'Other_well_location', 'Coordinates_2', 'nine',
       'ten_one', 'ten_two'],
      dtype='object')

In [123]:
df_clean = df.iloc[:,-13:]
df_original = df.iloc[:,:-13]

In [77]:
## Testing if aggregating makes things better
df3 = df['firm_name'].dropna()
len(df3)

1048575

# Now we have a clean data set: what does it look like?

In [125]:
df_clean

,tract_owner,tract_id,ID_Date_B,firm_name,Survey,Coordinates,Cities,Well_location,Other_well_location,Coordinates_2,nine,ten_one,ten_two
0,None,1200219870303,0,,,,NaN,NaN,NaN,,NaN,NaN,NaN
1,"O'CONNOR, T. J. (JIM)",01000000199175,02227525000019770606,"DOW CHEMICAL CO., THE",,,NaN,NaN,NaN,,NaN,NaN,NaN
2,"O'CONNOR, T. J. (JIM)",02000000199175,0210411022752514,4,IGNACIO GUAJARDO SUR. A-127,128800000800NE,BERCLAIR,660 FSL - 4849 FWL,660 FSL - 4849 FWL,"2950',2700',3000'",0.0,NNN010229394,17501047
3,,0319028076,0,,,,NaN,NaN,NaN,,NaN,NaN,NaN
4,STATE OF TEXAS TRACT 296,01000000299321,03313100000019770606,GOLDEN TREND OIL & GAS CORP.,,,NaN,NaN,NaN,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,,0330655400,0,,,,NaN,NaN,NaN,,NaN,NaN,NaN
1048571,None,06010933120,0,,,,NaN,NaN,NaN,,NaN,NaN,NaN
1048572,None,0602,0,,,,NaN,NaN,NaN,,NaN,NaN,NaN
1048573,None,060301750,0,,,,NaN,NaN,NaN,,NaN,NaN,NaN


In [142]:
uniques = []
for i in list(df_clean.columns):
    x = df_clean[i].unique()
    uniques.append(len(x))

In [143]:
uniques

[87003,
 437145,
 314909,
 19030,
 137287,
 12827,
 4424,
 126145,
 149488,
 19844,
 2,
 20849,
 177141]

In [144]:
df

,1200119870303W-1 FILED AS AN AMENDED NEW STATU,S NUMBER ASSIGNED IN ERROR. REFER TO 0000000000,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,firm_name,Survey,Coordinates,Cities,Well_location,Other_well_location,Coordinates_2,nine,ten_one,ten_two
0,1200219870303PERMIT 324685. (BJM) 03/03/87,0,NaN,nan,nan,NaN,NaN,NaN,nan,nan,...,,,,NaN,NaN,NaN,,NaN,NaN,NaN
1,"01000000199175O'CONNOR, T. J. (JIM)","02227525000019770606DOW CHEMICAL CO., THE",00ANNNNNNNNNNN00000011977060600000000N ...,nan,nan,NaN,NaN,NaN,nan,nan,...,"DOW CHEMICAL CO., THE",,,NaN,NaN,NaN,,NaN,NaN,NaN
2,"02000000199175O'CONNOR, T. J. (JIM)",021 0411022752514,0564337702700000013880001977060619770606000000...,NNNN00000000000000OIGNACIO GUAJARDO SUR. A-127,00000000000000000000000000000000128800000800NE,BERCLAIR,660 FSL - 4849 FWL,00000000000000660 FSL - 4849 FWL,"000000000000002950',2700',3000'",O00000000,...,4,IGNACIO GUAJARDO SUR. A-127,128800000800NE,BERCLAIR,660 FSL - 4849 FWL,660 FSL - 4849 FWL,"2950',2700',3000'",0.0,NNN010229394,17501047
3,0319028076GG N00000000197703040000000000000000,0,NaN,nan,nan,NaN,NaN,NaN,nan,nan,...,,,,NaN,NaN,NaN,,NaN,NaN,NaN
4,01000000299321STATE OF TEXAS TRACT 296,03313100000019770606GOLDEN TREND OIL & GAS CORP.,00ANNNNNNNNNNN00000021977060600000000N ...,nan,nan,NaN,NaN,NaN,nan,nan,...,GOLDEN TREND OIL & GAS CORP.,,,NaN,NaN,NaN,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,0330655400OO N00000000198307240000000000000000,0,NaN,nan,nan,NaN,NaN,NaN,nan,nan,...,,,,NaN,NaN,NaN,,NaN,NaN,NaN
1048571,0601D 0933 1200 P0000,0,NaN,nan,nan,NaN,NaN,NaN,nan,nan,...,,,,NaN,NaN,NaN,,NaN,NaN,NaN
1048572,0602C P0000,0,NaN,nan,nan,NaN,NaN,NaN,nan,nan,...,,,,NaN,NaN,NaN,,NaN,NaN,NaN
1048573,0603W 01750 P0000,0,NaN,nan,nan,NaN,NaN,NaN,nan,nan,...,,,,NaN,NaN,NaN,,NaN,NaN,NaN
